In [1]:
seed = 42

In [5]:
!pip install torch --index-url https://download.pytorch.org/whl/cpu

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cpu


In [6]:
!pip install scikit-learn pandas numpy==1.26.0 scipy pyarrow

Defaulting to user installation because normal site-packages is not writeable


In [7]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch.utils.data import DataLoader, TensorDataset

In [8]:
agent_info = pd.read_parquet('nus_agent_info_df.parquet')
client_info = pd.read_parquet('nus_client_info_df.parquet')
policy_info = pd.read_parquet('nus_policy_info_df.parquet')

In [9]:
display(client_info.head())

,secuityno,cltsex,cltdob,marryd,race_desc_map,cltpcode,household_size,economic_status,family_size,household_size_grp,family_size_grp
0,CIN:14264,F,1993-02-17,M,Chinese,545686,1,76,56,HH1_lt40,FS3_40to60
1,CIN:7188,F,1977-06-15,M,Chinese,308364,72,96,90,HH2_40to80,FS5_80up
2,CIN:13608,F,1998-02-12,S,Chinese,387393,28,93,23,HH1_lt40,FS2_20to40
3,CIN:5087,F,1972-10-25,M,Chinese,640469,84,51,34,HH3_80to100,FS2_20to40
4,CIN:18531,M,1984-12-27,M,Others,763318,92,18,73,HH3_80to100,FS4_60to80


In [10]:
display(policy_info.head())

,chdrnum,agntnum,secuityno,occdate,annual_premium,product,flg_main,flg_rider,flg_inforce,flg_lapsed,flg_cancel,flg_expire,flg_converted,product_grp,cust_age_at_purchase_grp,cust_tenure_at_purchase_grp
0,PID:281,AIN:62,CIN:6957,2018-11-12,0.0,prod_8,1,0,1,0,0,0,1,PG:0,AG05_35to39,TNR2_lt1yr
1,PID:280,AIN:63,CIN:2161,2024-02-22,7.0,prod_8,1,0,1,0,0,0,1,PG:0,AG04_30to34,TNR4_4to8yr
2,PID:2577,AIN:63,CIN:16605,2024-08-28,423.0,prod_6,1,0,1,0,0,0,1,PG:0,AG08_50to54,TNR5_8yrup
3,PID:2578,AIN:63,CIN:16605,2024-08-27,217.0,prod_6,1,0,1,0,0,0,1,PG:0,AG08_50to54,TNR5_8yrup
4,PID:305,AIN:63,CIN:7917,2024-08-28,432.0,prod_6,1,0,1,0,0,0,1,PG:0,AG07_45to49,TNR4_4to8yr


In [11]:
display(agent_info.head())

,agntnum,agent_age,agent_gender,agent_marital,agent_tenure,cnt_converted,annual_premium_cnvrt,pct_lapsed,pct_cancel,pct_inforce,...,pct_AG03_25to29,pct_AG04_30to34,pct_AG05_35to39,pct_AG06_40to44,pct_AG07_45to49,pct_AG08_50to54,pct_AG09_55to59,pct_AG10_60up,cluster,agent_product_expertise
0,AIN:9513,32.0,M,M,15.0,21.0,1.004900e+04,1.0,0.814954,0.000000,...,0.142857,0.000000,0.000000,0.000000,0.000000,0.238095,0.142857,0.238095,3,"[prod_2, prod_6]"
1,AIN:4310,40.0,M,M,18.0,396.0,1.971080e+05,0.6,0.083937,0.429293,...,0.131313,0.388889,0.116162,0.184343,0.083333,0.020202,0.040404,0.010101,6,[prod_6]
2,AIN:4302,39.0,F,M,40.0,853.0,5.106351e+05,0.3,0.017406,0.711606,...,0.137163,0.144197,0.177022,0.311841,0.106682,0.036342,0.035170,0.010551,6,[prod_4]
3,AIN:4996,57.0,F,D,41.0,554.0,3.514724e+05,0.3,0.003623,0.584838,...,0.009025,0.128159,0.108303,0.234657,0.162455,0.135379,0.160650,0.037906,6,"[prod_7, prod_9, prod_6, prod_0, prod_2]"
4,AIN:3457,38.0,M,M,62.0,1525.0,1.215380e+06,0.3,0.017523,0.640656,...,0.069508,0.201311,0.215082,0.214426,0.108197,0.084590,0.043279,0.015082,6,"[prod_6, prod_8]"


In [12]:
client_info['yob'] = client_info['cltdob'].dt.year
client_info_dropped = client_info.drop(columns=["cltdob", "cltpcode", "household_size_grp", "family_size_grp"])
client_info_dropped = client_info_dropped[client_info_dropped['marryd'] != 'P']
client_info_dropped.head()

,secuityno,cltsex,marryd,race_desc_map,household_size,economic_status,family_size,yob
0,CIN:14264,F,M,Chinese,1,76,56,1993.0
1,CIN:7188,F,M,Chinese,72,96,90,1977.0
2,CIN:13608,F,S,Chinese,28,93,23,1998.0
3,CIN:5087,F,M,Chinese,84,51,34,1972.0
4,CIN:18531,M,M,Others,92,18,73,1984.0


In [80]:
from sklearn.preprocessing import MultiLabelBinarizer

agent_info = agent_info[agent_info['agent_gender'] != 'U']

mlb = MultiLabelBinarizer()
encoded_products = mlb.fit_transform(agent_info['agent_product_expertise'])
df_encoded_products = pd.DataFrame(encoded_products, columns=mlb.classes_, dtype=object)

mlb = MultiLabelBinarizer()
encoded_genders = mlb.fit_transform(agent_info['agent_gender'])
df_encoded_genders = pd.DataFrame(encoded_genders, columns=mlb.classes_, dtype=object)
df_encoded_genders.rename(columns={'M':'gender_M','F':'gender_F'},inplace=True)

mlb = MultiLabelBinarizer()
encoded_marital = mlb.fit_transform(agent_info['agent_marital'])
df_encoded_marital = pd.DataFrame(encoded_marital, columns=mlb.classes_, dtype=object)

agent_info_encoded = pd.concat([agent_info, df_encoded_products, df_encoded_genders, df_encoded_marital], axis=1).drop(columns=['agent_product_expertise', 'pct_SX0_unknown', 'cluster','agent_gender','agent_marital'])

agent_info_encoded.drop(columns=['pct_prod_0_cnvrt','pct_prod_1_cnvrt','pct_prod_2_cnvrt','pct_prod_3_cnvrt', 'pct_prod_4_cnvrt','pct_prod_5_cnvrt'])
agent_info_encoded.head()

,agntnum,agent_age,agent_tenure,cnt_converted,annual_premium_cnvrt,pct_lapsed,pct_cancel,pct_inforce,pct_prod_0_cnvrt,pct_prod_1_cnvrt,...,prod_7,prod_8,prod_9,gender_F,gender_M,D,M,S,U,W
0,AIN:9513,32.0,15.0,21.0,1.004900e+04,1.0,0.814954,0.000000,0.0,0.0,...,0,0,0,0,1,0,1,0,0,0
1,AIN:4310,40.0,18.0,396.0,1.971080e+05,0.6,0.083937,0.429293,0.0,0.0,...,0,0,0,0,1,0,1,0,0,0
2,AIN:4302,39.0,40.0,853.0,5.106351e+05,0.3,0.017406,0.711606,0.0,0.0,...,0,0,0,1,0,0,1,0,0,0
3,AIN:4996,57.0,41.0,554.0,3.514724e+05,0.3,0.003623,0.584838,0.0,0.0,...,1,0,1,1,0,1,0,0,0,0
4,AIN:3457,38.0,62.0,1525.0,1.215380e+06,0.3,0.017523,0.640656,0.0,0.0,...,0,1,0,0,1,0,1,0,0,0


In [81]:
merged_df = policy_info.merge(client_info_dropped, on="secuityno").merge(agent_info_encoded, on="agntnum")
merged_df.dropna(inplace=True)

merged_df["cust_age_at_purchase"] = merged_df["occdate"].dt.year - merged_df["yob"]

merged_df = merged_df.drop(columns=["chdrnum", "secuityno", "agntnum","occdate", "flg_main", "flg_rider", "flg_cancel", "flg_converted", "flg_inforce", "flg_expire", "flg_lapsed"])
print(merged_df.describe())

display(merged_df.head())

       annual_premium           yob     agent_age  agent_tenure  \
count    28933.000000  28933.000000  28933.000000  28933.000000   
mean      1073.198993   1978.661148     39.545121     62.519787   
std       4382.362989     10.670054      9.406474     56.920644   
min          0.000000   1940.000000     22.000000      0.000000   
25%          0.000000   1972.000000     33.000000     22.000000   
50%        120.000000   1980.000000     37.000000     42.000000   
75%        525.000000   1987.000000     44.000000     84.000000   
max     300039.600000   2006.000000     87.000000    427.000000   

       cnt_converted  annual_premium_cnvrt    pct_lapsed    pct_cancel  \
count   28933.000000          2.893300e+04  28933.000000  28933.000000   
mean     1160.433381          6.479471e+05      0.249501      0.106535   
std      2422.155661          1.913596e+06      0.289880      0.128704   
min         1.000000          0.000000e+00      0.000000      0.000000   
25%       118.000000      

,annual_premium,product,product_grp,cust_age_at_purchase_grp,cust_tenure_at_purchase_grp,cltsex,marryd,race_desc_map,household_size,economic_status,...,prod_8,prod_9,gender_F,gender_M,D,M,S,U,W,cust_age_at_purchase
0,0.0,prod_8,PG:0,AG05_35to39,TNR2_lt1yr,F,M,Chinese,25,78,...,0,1,0,1,0,0,1,0,0,36.0
1,7.0,prod_8,PG:0,AG04_30to34,TNR4_4to8yr,M,S,Chinese,57,31,...,0,1,0,1,0,0,1,0,0,33.0
2,423.0,prod_6,PG:0,AG08_50to54,TNR5_8yrup,M,M,Chinese,56,30,...,0,1,0,1,0,0,1,0,0,51.0
3,217.0,prod_6,PG:0,AG08_50to54,TNR5_8yrup,M,M,Chinese,56,30,...,0,1,0,1,0,0,1,0,0,51.0
4,432.0,prod_6,PG:0,AG07_45to49,TNR4_4to8yr,F,M,Malay,23,00,...,0,1,0,1,0,0,1,0,0,46.0


In [97]:
import warnings
import sklearn as skl
from sklearn.preprocessing import LabelEncoder, StandardScaler
warnings.filterwarnings('ignore')

client_features = list(client_info_dropped.columns.drop(['secuityno', 'yob']))
client_features.append('cust_age_at_purchase')
#agent_targets = agent_info.columns.drop(['agntnum', 'agent_product_expertise', 'pct_SX0_unknown', 'cluster'])
agent_targets = agent_info_encoded.columns.drop(['agntnum'])

X = merged_df[client_features]
y = merged_df[agent_targets]
display(y.head)

#X_classification_columns = X.select_dtypes(include=['object', 'category']).columns
#X_regression_columns = X.select_dtypes(include=['number']).columns
y_classification_columns = y.select_dtypes(include=['object', 'category']).columns
#y_classification_columns = ['prod_0','prod_2','prod_4','prod_6','prod_7','prod_8','prod_9','gender_F','gender_M','D','M','S','U','W']
y_regression_columns = y.select_dtypes(include=['number']).columns

#X_classification = X[X_classification_columns]
#X_regression = X[X_regression_columns]
y_classification = y[y_classification_columns]
y_regression = y[y_regression_columns]

X_encoder = LabelEncoder()
for col in X.select_dtypes(include=["object"]).columns:
    X[col] = X_encoder.fit_transform(X[col])
display(X.head())

X_scaler = StandardScaler()
X = X_scaler.fit_transform(X)

y_encoder = LabelEncoder()
for col in y_classification.select_dtypes(include=["object"]).columns:
    y_classification[col] = y_encoder.fit_transform(y_classification[col])
display(y_classification.head())

y_scaler = StandardScaler()
y_regression = y_scaler.fit_transform(y_regression) # classification only or regression too?
# just look at classification

# 70% training data, 10% validation data, 20% test data
X_c_train, X_c_temp, y_c_train_all, y_c_temp_all = skl.model_selection.train_test_split(X, y_classification, test_size=0.3, 
                                                                                        random_state=seed)
X_c_val, X_c_test, y_c_val_all, y_c_test_all = skl.model_selection.train_test_split(X_c_temp, y_c_temp_all, test_size=2/3, random_state=seed)
X_r_train, X_r_temp, y_r_train_all, y_r_temp_all = skl.model_selection.train_test_split(X, y_regression, test_size=0.3, random_state=seed)
X_r_val, X_r_test, y_r_val_all, y_r_test_all = skl.model_selection.train_test_split(X_r_temp, y_r_temp_all, test_size=2/3, random_state=seed)

<bound method NDFrame.head of        agent_age  agent_tenure  cnt_converted  annual_premium_cnvrt  \
0           31.0           6.0           47.0              6112.000   
1           32.0          71.0         4560.0           1045929.756   
2           32.0          71.0         4560.0           1045929.756   
3           32.0          71.0         4560.0           1045929.756   
4           32.0          71.0         4560.0           1045929.756   
...          ...           ...            ...                   ...   
29480       39.0          24.0          476.0            569045.700   
29481       52.0           7.0           32.0             13780.000   
29482       70.0         216.0          195.0             98906.500   
29483       48.0          28.0           19.0             10779.000   
29484       34.0          13.0            1.0               701.000   

       pct_lapsed  pct_cancel  pct_inforce  pct_prod_0_cnvrt  \
0             0.1    0.243689     0.936170          0

,cltsex,marryd,race_desc_map,household_size,economic_status,family_size,cust_age_at_purchase
0,0,1,0,167,78,98,36.0
1,1,2,0,301,31,47,33.0
2,1,1,0,300,30,60,51.0
3,1,1,0,300,30,60,51.0
4,0,1,2,146,0,55,46.0


,prod_0,prod_2,prod_4,prod_6,prod_7,prod_8,prod_9,gender_F,gender_M,D,M,S,U,W
0,1,1,0,0,0,0,1,0,1,0,0,1,0,0
1,1,0,1,1,0,0,1,0,1,0,0,1,0,0
2,1,0,1,1,0,0,1,0,1,0,0,1,0,0
3,1,0,1,1,0,0,1,0,1,0,0,1,0,0
4,1,0,1,1,0,0,1,0,1,0,0,1,0,0


In [84]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=25):
    model.train()
    for epoch in range(epochs):
        train_loss = 0.0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
                
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        # Validation Phase
        model.eval()
        val_loss = 0.0
        with torch.inference_mode():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                y_pred = model(X_batch)
                val_loss += criterion(y_pred, y_batch).item()
            model.train()
        
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss/len(train_loader):.4f} - Val Loss: {val_loss/len(val_loader):.4f}")

In [16]:
class AgentPredictionRegNN(nn.Module):
    def __init__(self, input_dim):
        super(AgentPredictionRegNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),  # Normalise activations
            nn.LeakyReLU(0.01),
            nn.Dropout(0.5),

            nn.Linear(256, 32),
            nn.LeakyReLU(0.01),

            nn.Linear(32, 1)  # For regression (Remove activation here)
        )

    def forward(self, x):
        logits = self.model(x)
        return logits
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AgentPredictionRegNN(input_dim=X.shape[1]).to(device)
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.02, weight_decay=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [19]:
models_r = []

for idx in range(len(y_r_train_all[0])):
    y_train = y_r_train_all[:,idx]
    y_val = y_r_val_all[:,idx]
    X_train, y_train = torch.tensor(X_r_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
    X_val, y_val = torch.tensor(X_r_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32)
    
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=32)
    
    print("X_train shape:", X_train.shape)
    print("y_train shape:", y_train.shape)
    
    model = AgentPredictionRegNN(input_dim=X.shape[1]).to(device)
    train_model(model, train_loader, val_loader, nn.MSELoss(), optimizer, epochs=10)
    models_r.append(model)

X_train shape: torch.Size([20275, 7])
y_train shape: torch.Size([20275])
Epoch 1/10 - Train Loss: 1.0349 - Val Loss: 1.0408
Epoch 2/10 - Train Loss: 1.0351 - Val Loss: 1.0407
Epoch 3/10 - Train Loss: 1.0339 - Val Loss: 1.0414
Epoch 4/10 - Train Loss: 1.0348 - Val Loss: 1.0413
Epoch 5/10 - Train Loss: 1.0337 - Val Loss: 1.0415
Epoch 6/10 - Train Loss: 1.0356 - Val Loss: 1.0406
Epoch 7/10 - Train Loss: 1.0350 - Val Loss: 1.0409
Epoch 8/10 - Train Loss: 1.0353 - Val Loss: 1.0411
Epoch 9/10 - Train Loss: 1.0354 - Val Loss: 1.0410
Epoch 10/10 - Train Loss: 1.0350 - Val Loss: 1.0421
X_train shape: torch.Size([20275, 7])
y_train shape: torch.Size([20275])
Epoch 1/10 - Train Loss: 1.0529 - Val Loss: 0.9669
Epoch 2/10 - Train Loss: 1.0518 - Val Loss: 0.9661
Epoch 3/10 - Train Loss: 1.0508 - Val Loss: 0.9680
Epoch 4/10 - Train Loss: 1.0520 - Val Loss: 0.9640
Epoch 5/10 - Train Loss: 1.0501 - Val Loss: 0.9673
Epoch 6/10 - Train Loss: 1.0526 - Val Loss: 0.9684
Epoch 7/10 - Train Loss: 1.0513 - Val

Epoch 1/10 - Train Loss: 1.0953 - Val Loss: 0.7238
Epoch 2/10 - Train Loss: 1.0760 - Val Loss: 0.7235
Epoch 3/10 - Train Loss: 1.0747 - Val Loss: 0.7254
Epoch 4/10 - Train Loss: 1.0761 - Val Loss: 0.7238
Epoch 5/10 - Train Loss: 1.0759 - Val Loss: 0.7243
Epoch 6/10 - Train Loss: 1.0759 - Val Loss: 0.7235
Epoch 7/10 - Train Loss: 1.0790 - Val Loss: 0.7253
Epoch 8/10 - Train Loss: 1.0759 - Val Loss: 0.7238
Epoch 9/10 - Train Loss: 1.0855 - Val Loss: 0.7253
Epoch 10/10 - Train Loss: 1.0759 - Val Loss: 0.7228
X_train shape: torch.Size([20275, 7])
y_train shape: torch.Size([20275])
Epoch 1/10 - Train Loss: 1.0458 - Val Loss: 1.0397
Epoch 2/10 - Train Loss: 1.0459 - Val Loss: 1.0453
Epoch 3/10 - Train Loss: 1.0461 - Val Loss: 1.0440
Epoch 4/10 - Train Loss: 1.0457 - Val Loss: 1.0420
Epoch 5/10 - Train Loss: 1.0466 - Val Loss: 1.0449
Epoch 6/10 - Train Loss: 1.0457 - Val Loss: 1.0460
Epoch 7/10 - Train Loss: 1.0448 - Val Loss: 1.0450
Epoch 8/10 - Train Loss: 1.0453 - Val Loss: 1.0453
Epoch 9/

Epoch 2/10 - Train Loss: 1.0290 - Val Loss: 1.2534
Epoch 3/10 - Train Loss: 1.0295 - Val Loss: 1.2561
Epoch 4/10 - Train Loss: 1.0295 - Val Loss: 1.2556
Epoch 5/10 - Train Loss: 1.0285 - Val Loss: 1.2551
Epoch 6/10 - Train Loss: 1.0296 - Val Loss: 1.2563
Epoch 7/10 - Train Loss: 1.0295 - Val Loss: 1.2552
Epoch 8/10 - Train Loss: 1.0292 - Val Loss: 1.2547
Epoch 9/10 - Train Loss: 1.0293 - Val Loss: 1.2557
Epoch 10/10 - Train Loss: 1.0291 - Val Loss: 1.2542


In [99]:
class AgentPredictionClassNN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(AgentPredictionClassNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),  # Normalize activations
            nn.LeakyReLU(0.01),
            nn.Dropout(0.5),

            nn.Linear(256, 32),
            nn.LeakyReLU(0.01),

            nn.Linear(32, num_classes)  # Output logits for each class
        )

    def forward(self, x):
        logits = self.model(x)
        return logits

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AgentPredictionClassNN(input_dim=X.shape[1], num_classes=len(np.unique(X))).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [100]:
models_c = []

for idx in y_c_train_all.columns:
    y_train = y_c_train_all[idx].values  # Convert to numpy array
    y_val = y_c_val_all[idx].values  # Convert to numpy array
    X_train, y_train = torch.tensor(X_c_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long)
    X_val, y_val = torch.tensor(X_c_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long)
    
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=32)
    
    print("X_train shape:", X_train.shape)
    print("y_train shape:", y_train.shape)
    
    model = AgentPredictionClassNN(input_dim=X.shape[1], num_classes=len(np.unique(y_train))).to(device)
    train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10)
    models_c.append(model)

X_train shape: torch.Size([20253, 7])
y_train shape: torch.Size([20253])
Epoch 1/10 - Train Loss: 0.6930 - Val Loss: 0.6894
Epoch 2/10 - Train Loss: 0.6927 - Val Loss: 0.6892
Epoch 3/10 - Train Loss: 0.6936 - Val Loss: 0.6892
Epoch 4/10 - Train Loss: 0.6927 - Val Loss: 0.6891
Epoch 5/10 - Train Loss: 0.6931 - Val Loss: 0.6894
Epoch 6/10 - Train Loss: 0.6952 - Val Loss: 0.6890
Epoch 7/10 - Train Loss: 0.6946 - Val Loss: 0.6893
Epoch 8/10 - Train Loss: 0.6931 - Val Loss: 0.6897
Epoch 9/10 - Train Loss: 0.6937 - Val Loss: 0.6891
Epoch 10/10 - Train Loss: 0.6940 - Val Loss: 0.6891
X_train shape: torch.Size([20253, 7])
y_train shape: torch.Size([20253])
Epoch 1/10 - Train Loss: 0.7238 - Val Loss: 0.7019
Epoch 2/10 - Train Loss: 0.7216 - Val Loss: 0.7027
Epoch 3/10 - Train Loss: 0.7221 - Val Loss: 0.7018
Epoch 4/10 - Train Loss: 0.7230 - Val Loss: 0.7017
Epoch 5/10 - Train Loss: 0.7208 - Val Loss: 0.7015
Epoch 6/10 - Train Loss: 0.7228 - Val Loss: 0.7014
Epoch 7/10 - Train Loss: 0.7216 - Val

In [22]:
# For regression NN
ys_reg = []

for idx in range(len(y_r_train_all[0])):
    y_test = y_r_test_all[:, idx]
    X_test, y_test = torch.tensor(X_r_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)
    
    test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32)
    
    model_reg = models_r[idx]
    y_idx = []
    
    model_reg.eval()
    test_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model_reg(X_batch)
            y_idx.append(y_pred.cpu().numpy().flatten())
            test_loss += nn.MSELoss()(y_pred, y_batch).item()
    print(f"Final Test Loss: {test_loss/len(test_loader):.4f}")
    ys_reg.append(np.concatenate(y_idx))
ys_reg = np.array(ys_reg).T
y_original_reg = y_scaler.inverse_transform(ys_reg).T

Final Test Loss: 1.0054
Final Test Loss: 1.0629
Final Test Loss: 1.1000
Final Test Loss: 1.0953
Final Test Loss: 1.0028
Final Test Loss: 1.0386
Final Test Loss: 1.0275
Final Test Loss: 0.7223
Final Test Loss: 0.9706
Final Test Loss: 1.0437
Final Test Loss: 1.0968
Final Test Loss: 0.9995
Final Test Loss: 0.8703
Final Test Loss: 1.1197
Final Test Loss: 1.1003
Final Test Loss: 1.0268
Final Test Loss: 1.0222
Final Test Loss: 0.9894
Final Test Loss: 1.0746
Final Test Loss: 0.8492
Final Test Loss: 0.8945
Final Test Loss: 1.0159
Final Test Loss: 1.0093
Final Test Loss: 1.0510
Final Test Loss: 1.0602
Final Test Loss: 0.9447
Final Test Loss: 1.0762
Final Test Loss: 0.9198
Final Test Loss: 0.9322


In [102]:
# For classification NN
ys_class = []

for idx in range(len(list(y_c_test_all.columns))):
    y_test = y_c_test_all.iloc[:, idx].values
    X_test, y_test = torch.tensor(X_c_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long)  # Change to long type
    
    test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32)
    
    model_class = models_c[idx]  # Adjust index to get the correct model
    y_idx = []
    
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch)
            y_pred = y_pred.view(-1, y_pred.size(-1))  # Ensure y_pred has shape (N, C)
            test_loss += nn.CrossEntropyLoss()(y_pred, y_batch).item()
            #y_idx.append(y_pred.cpu().numpy().flatten())
            y_idx.extend(y_pred.cpu().numpy().tolist())  # Use extend instead of append
    print(f"Final Test Loss: {test_loss/len(test_loader):.4f}")
    y_idx = np.array(y_idx).argmax(axis=1)  # Get the class with the highest probability
    ys_class.append(y_idx)  # Append y_idx to ys_class
    #ys_class.append(np.concatenate(y_idx))
    
ys_class = np.array(ys_class)

# Fit the y_encoder with all possible labels
all_labels = np.concatenate([y_c_train_all.values.flatten(), y_c_test_all.values.flatten()])
y_encoder.fit(all_labels)

# Decode the classification predictions
y_decoded = [y_encoder.inverse_transform(ys_class[:, i]) for i in range(ys_class.shape[1])]

Final Test Loss: 0.6962
Final Test Loss: 0.6960
Final Test Loss: 0.6968
Final Test Loss: 0.6969
Final Test Loss: 0.6955
Final Test Loss: 0.6966
Final Test Loss: 0.6957
Final Test Loss: 0.6959
Final Test Loss: 0.6927
Final Test Loss: 0.7098
Final Test Loss: 0.6952
Final Test Loss: 0.6984
Final Test Loss: 0.7072
Final Test Loss: 0.7107


In [130]:
def perf(y_true_reg, y_pred_reg, y_true_class, y_pred_class):
    y_true_class = np.array(y_true_class)
    success = 0
    for col_reg in range(y_true_reg.shape[1]):
        for row_reg in range(y_true_reg.shape[0]):
            if np.abs(y_true_reg[row_reg, col_reg] - y_pred_reg[row_reg, col_reg]) / y_true_reg[row_reg, col_reg] < 0.025:
                success += 1
    #for col_class in range(y_true_class.shape[1]):
        #print(y_true_class[:, col_class])
        #print(y_pred_class[:, col_class])
        #success += np.sum(y_true_class[:, col_class] == y_pred_class[:, col_class])
    success += np.sum(np.sum(y_true_class == y_pred_class.T, axis=1))
    
    return(success)

In [131]:
# Ensure performance is not redefined as an integer
p = perf(y_r_test_all, ys_reg, y_c_test_all, ys_class) / (y_r_test_all.shape[0])
print(p)

26.197857266286505
